In [2]:
import requests
import re
import os
import pickle

from bs4 import BeautifulSoup
from tqdm import tqdm

In [9]:
# サーチバー上のtags以降のtag文字列を参考にする.
# ex: https://safebooru.org/index.php?page=post&s=list&tags=kizuna_ai
search_tag    = "kizuna_ai"
page_unit     = 40 #1ページに40枚ずつ表示される.
all_page_num  = 29 #tagが持つページ数 (自動で取得できそう.)
base_url      = "https://safebooru.org/index.php?page=post&s=list&tags={}&pid={}"

### 取得対象の一覧取得

In [10]:
all_image_thumb_elem_list = []
#各ページを確認しながら、画像のurlの要素を取得する
for page_num in tqdm(range(all_page_num)):
    pid = page_unit * page_num
    page_url_pid = base_url.format(search_tag,pid)
    responce = requests.get(page_url_pid)
    soup     = BeautifulSoup(responce.text)
    page_preview_list  = soup.find_all(class_="thumb")
    all_image_thumb_elem_list += page_preview_list

100%|██████████| 29/29 [00:28<00:00,  1.01it/s]


In [13]:
#取得対象の画像のurl一覧を取得する.
all_image_content_url_list = []
for image_thumb_elem in all_image_thumb_elem_list:
    assert len( image_thumb_elem.contents ) == 1, "thumbのcontent要素が1つではありません."    
    image_ref = image_thumb_elem.contents[0]["href"]
    image_url = f"https://safebooru.org/{image_ref}"
    all_image_content_url_list.append(image_url)

In [14]:
print(f"{len(all_image_content_url_list)}件の画像urlを取得しました.")
with open(os.path.join("cache","all_image_content_url_list.pkl"),"wb") as f:
    pickle.dump(all_image_content_url_list,f)

1155件の画像urlを取得しました.


### 画像とメタデータの取得

In [3]:
from scraping_from_safebooru import get_content_and_metainfo_from_image_url

#urlリストの取得
with open(os.path.join("cache","all_image_content_url_list.pkl"),"rb") as f:
    all_image_content_url_list = pickle.load(f)

In [4]:
for image_content_url in tqdm(all_image_content_url_list):
    get_content_and_metainfo_from_image_url(image_content_url,save_dir="save",debug_mode=False)

  9%|▉         | 105/1155 [04:23<48:40,  2.78s/it] 

In [5]:
#Todo: 各画像の取得処理を並列化させる.
#全然並列処理になってくれない. 
#https://qiita.com/tchnkmr/items/b05f321fa315bbce4f77
from concurrent.futures import ThreadPoolExecutor
thread_pool_executor = ThreadPoolExecutor(max_workers=10)

for image_content_url in all_image_content_url_list:
    thread_pool_executor.submit(
        get_content_and_metainfo_from_image_url(image_content_url,save_dir="save")
    )